In [37]:
"""do necessary modifications to 4W file"""

from collections import OrderedDict

import numpy as np
import pandas as pd
import country_converter as coco
import dateparser

pd.set_option('display.max_columns', 500)

In [174]:
xmet = pd.read_excel('../d0cz/w_overview.xlsx', sheet_name='merged')

In [175]:
xed = pd.read_excel('../d0cz/4w_content_final.xlsx', sheet_name='Sheet1')

In [176]:
#clean up xed
xed = xed.rename(
    {'File' : 'file_nm',
    'Date Reported' : 'date_rep',
    'Donor' : 'donor',
    'Organisation' : 'org',
    'Implementing Partner' : 'imp_partner',
    'Activity category' : 'act_cat',
    'Area of activities' : 'area_act',
    'Activity detail' : 'act_detail',
    'Admin level 1' : 'admin_lvl_1',
    'Admin level 1 code' : 'admin_lvl_1_cd',
    'Admin level 2' : 'admin_lvl_2',
    'Admin level 2 code' : 'admin_lvl_2_cd',
    'Status' : 'status',
    '# of HH reached' : 'num_hh_reached',
    'Number of beneficiaries reached' : 'num_ben_reached',
    'Start date' : 'start_dt',
    'End date' : 'end_dt'}, axis = 1)

xed.file_nm = xed.apply(lambda x : x['file_nm'].split(':')[1], axis = 1)

In [177]:
#do things to xmet

#fix cols
xmet = xmet.rename(
    {
    'Filename': 'file_nm',
    'Crisis Name (if available)' : 'crisis_nm',
    'Country' : 'country',
    'Year' : 'year',
    'Extracted URL' : 'url',
    'Found N/Y' : 'found_yn',
    'Multi Sector?' : 'mult',
    'Multi Sector Shelter Name' : 'mult_nm',
    'Data Sheet' : 'sht',
    'notes' : 'notes',
    '3Ws' : '3w',
    '4Ws' : '4w',
    '5Ws' : '5w',
    '4W Overview' : 'over'
    }, axis = 1
)

#fix names

misp = \
    {'Aghanistan' : 'Afghanistan',
    'Bnagladesh' : 'Bangladesh',
    'Ethopia' : 'Ethiopia',
    'Phillipines' : 'Philippines',
    'El Salvado' : 'El Salvador',
    'Equador' : 'Ecuador',
    'DRC' : 'Democratic Repulic of Congo',
    'CAR' : 'Central African Republic'}

for k,v in misp.items():
    xmet.country[xmet.country == k] = v

xmet['iso'] = xmet['country'].apply(lambda x : coco.convert(names = x, to = 'ISO3'))

xmet['uid'] = xmet.apply(lambda x : x['iso'] + str(x['year']), axis = 1)

xmet['country'] = xmet['country'].str.title()

/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [178]:
#merge and do some cleaning

def strip(s):
    if isinstance(s, str):
        return s.replace('\n', '').replace('\t', '').replace('.xlsx', '').replace('.xls', '') \
                .replace('.xlsm', '').replace(' ', '')

# xmet['mrg_v_met'] = xmet.apply(lambda x : strip(x.file_nm), axis = 1)
xed['mrg_v_ed'] = xed.apply(lambda x : strip(x.file_nm), axis = 1)

mrgd = xed.merge(xmet, on = 'file_nm', how = 'left')

mrgd[mrgd.iso.isna()].mrg_v_ed.unique()

repl = {
        'shelter_cccmdms_4w_january_2018' : 'NGA2018',
        '_2016_IRAQ_S_NFI_interventions_Jan._Jun.16_Cluster_4W_matrix_' : 'IRQ2016',
        'CRS-ActivityInfo2016-IRAQ_S_NFI_Cluster_4W_Template_Final.01' : 'IRQ2016',
        '_2016_iraq_s-nfi_interventions_jan._-_dec.16_cluster_4w_matrix_0' : 'IRQ2016',
        'CopyofAfg_WhoHasWhatWhere_Feb_2016-SUGGESTEDDATACOLLECTIONTEMPLATE' : 'AFG2016',
        '5W_Tableau_2017_v17' : 'UKR2017',
        'Ecuador 2016.xlsx' : 'ECU2016'
    }

for k,v in repl.items():
    mrgd.loc[mrgd.mrg_v_ed == k, 'uid'] = v
    
mrgd.num_hh_reached = mrgd.num_hh_reached.apply(lambda x : 0 if not (isinstance(x, float) \
                                                                     or isinstance(x, int)) else x)
mrgd.num_ben_reached = mrgd.num_ben_reached.apply(lambda x : 0 if not (isinstance(x, float)\
                                                                       or isinstance(x, int)) else x)
mrgd.org = mrgd.org.str.title()
mrgd.imp_partner = mrgd.imp_partner.str.title()

mrgd.loc[mrgd.org == 'United Nations High Comissioner For Refugee', 'org'] = 'Unhcr'
mrgd.loc[mrgd.org == 'United Nations High Comissioner For Refugees', 'org'] = 'Unhcr'
mrgd.loc[mrgd.org == 'International Organization For Migration', 'org'] = 'Iom'



In [43]:
# get rows without UID

type(list(mrgd.uid.unique())[3])

nolab = mrgd[~mrgd.uid.isin(list(mrgd.uid.unique()[1:]))]

for v in nolab.file_nm.unique():
    print(v)

Ecuador 2016.xlsx


# descriptions#

In [44]:
#total counts by crisis
r = mrgd.uid.value_counts()
r.to_clipboard()
r

UKR2017    69447
BGD2018    17654
SSD2015    13421
NPL2016     8915
YEM2016     2920
HTI2016     2128
IRQ2016     1973
YEM2015     1499
AFG2017     1331
AFG2016     1249
SYR2016     1059
ECU2016      890
AFG2013      795
PAK2015      760
BGD2017      428
NGA2017      323
IRQ2015      313
SYR2015      273
MLI2017      174
NGA2018       66
PSE2014       49
ETH2015        6
Name: uid, dtype: int64

In [45]:
# d = self.flows[['fts.uid', 'plan', 'amountUSD']].groupby(['fts.uid', 'plan'], as_index=False).aggregate(
#     {'amountUSD': 'sum'})

# piv = pd.pivot_table(d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[d['plan']] \
#                      , fill_value=0).reset_index()\


In [46]:
mrgd.head(10).to_clipboard()

In [47]:
#num reports by year
# fill_blank_num = mrgd.
d = mrgd[['year', 'country','file_nm', 'uid', 'num_hh_reached', 'num_ben_reached']].groupby(['year', 'country'])\
        .agg({'file_nm': 'nunique', 'country': 'count', 'num_hh_reached' : ['sum', 'median'], 'num_ben_reached' : ['sum', 'median']}).reset_index()

d.columns = ['_'.join(col) for col in d.columns.values]
    
d = d.rename({'uid' : 'total_count'}, axis=1)
d.sort_values('file_nm_nunique', ascending = False).to_clipboard()
d.sort_values('file_nm_nunique', ascending = False)
# pd.pivot_table(d, index='country', values='country', aggfunc=np.count_nonzero, columns=[d['year']]).reset_index()
# mrgd[['year', 'country']].drop_duplicates()

,year_,country_,file_nm_nunique,country_count,num_hh_reached_sum,num_hh_reached_median,num_ben_reached_sum,num_ben_reached_median
15,2017.0,Afghanistan,16,1331,1.116983e+05,43.0,6.667285e+06,222.0
8,2016.0,Afghanistan,8,559,1.332500e+04,22.0,1.974220e+05,80.0
11,2016.0,Iraq,7,1306,3.133200e+05,75.5,6.796000e+03,6796.0
19,2017.0,Ukraine,5,59341,1.646432e+06,8.0,5.563765e+06,13.0
16,2017.0,Bangladesh,3,428,5.124100e+04,200.0,2.614700e+05,1000.0
0,2013.0,Afghanistan,1,795,3.075200e+04,13.0,1.896080e+05,43.0
18,2017.0,Nigeria,1,323,4.932000e+04,100.0,0.000000e+00,NaN
17,2017.0,Mali,1,174,0.000000e+00,NaN,0.000000e+00,NaN
14,2016.0,Yemen,1,2920,3.409900e+04,61.5,2.162470e+05,0.0
13,2016.0,Syria,1,1059,7.643100e+04,40.0,4.692860e+05,157.0


In [48]:
mrgd.country.value_counts().to_clipboard()

In [49]:
mrgd.uid.value_counts().to_clipboard()

In [50]:
#reg len:125673

mrgd.uid.value_counts().to_clipboard()

In [51]:
#total count by org

# d = mrgd[['org','num_hh_reached', 'num_ben_reached']].groupby(['org'])\
#         .agg({'org' : 'count', 'num_hh_reached' : ['sum', 'mean'], 'num_ben_reached' : ['sum', 'mean']}).reset_index()

d = mrgd[['org','num_hh_reached', 'num_ben_reached']].groupby(['org'])\
        .agg({'org' : 'count', 'num_hh_reached' : ['sum'], 'num_ben_reached' : ['sum']}).reset_index()


d.columns = ['_'.join(col) for col in d.columns.values]

d = d.sort_values('org_count', ascending = False)

d.to_clipboard()

In [52]:
#PIN, y u so much?
print(mrgd[mrgd.org == 'People In Need'].uid.value_counts())

print(len(mrgd[mrgd.uid == 'UKR2017']))

UKR2017    21661
AFG2017       31
AFG2016        2
Name: uid, dtype: int64
69447


#### dates

In [53]:
#get unique start dts for merging
start_dt_fmt = [(x, dateparser.parse(x)) for x in list(set([v for v in mrgd.start_dt.values if isinstance(v, str)]))]

KeyboardInterrupt: 

In [ ]:
#get unique end dts for merging
end_dt_fmt = [(x, dateparser.parse(x)) for x in list(set([v for v in mrgd.end_dt.values if isinstance(v, str)]))]

In [ ]:
#get unique rep dts for merging
date_rep_fmt = [(x, dateparser.parse(x)) for x in list(set([v for v in mrgd.date_rep.values if isinstance(v, str)]))]

In [ ]:
#end: 8951 missing
len(mrgd[mrgd.apply(lambda x: isinstance(x.date_rep, str), axis = 1)])


#### merge formatted dates with missing dates

In [ ]:
#start: 8951 missing
len(mrgd[mrgd.apply(lambda x: isinstance(x.start_dt, str), axis = 1)])

#end: 109 missing
len(mrgd[mrgd.apply(lambda x: isinstance(x.end_dt, str), axis = 1)])

#start: 727 missing
len(mrgd[mrgd.apply(lambda x: isinstance(x.date_rep, str), axis = 1)])

In [ ]:
s_d = pd.DataFrame(start_dt_fmt, columns=['str', 'fmt'])

# mrgd.merge(s_d, left_on = 'start_dt', right_on='fmt')

def find(v):
    if v in s_d.str.values:
        return s_d[s_d.str == v].fmt
    else:
        return ''

# m.new = m.apply(lambda x : find(x.col1), axis = 1)

mrgd.start_dt = mrgd.apply(lambda x : find(x.start_dt), axis=1)

In [ ]:
#counts in same year?
r = mrgd.start_dt.head(500).apply(lambda x : dateparser.parse(x) if isinstance(x, str) else None)

In [58]:
#counts by dist type

d = mrgd[['act_detail','num_hh_reached', 'num_ben_reached']].groupby(['act_detail'])\
        .agg({'num_hh_reached' : ['sum'], 'num_ben_reached' : ['sum'], 'act_detail' : 'count'}).reset_index()

d.columns = ['_'.join(col) for col in d.columns.values]

d.sort_values('num_ben_reached_sum', ascending=False).to_excel('../d0cz/tmpacts.xlsx')

# mrgd.act_detail.value_counts()

In [179]:
#dist by country
t_mrgd = mrgd

d = t_mrgd[['act_detail','num_hh_reached', 'num_ben_reached', 'country', 'year']].groupby(['act_detail', 'country', 'year'])\
        .agg({'num_hh_reached' : ['sum'], 'num_ben_reached' : ['sum'], 'act_detail' : 'count'}).reset_index()

d.columns = ['_'.join(col) for col in d.columns.values]

d.sort_values('num_ben_reached_sum', ascending=False)

d.to_clipboard()

In [163]:
#gen count length

gen_d = mrgd[['act_detail','num_hh_reached', 'num_ben_reached']].groupby(['act_detail'])\
        .agg({'num_hh_reached' : ['sum'], 'num_ben_reached' : ['sum'], 'act_detail' : 'count'}).reset_index()

gen_d.columns = ['_'.join(col) for col in gen_d.columns.values]

gen_d.sort_values('num_ben_reached_sum', ascending=False)

len(gen_d)

522

In [106]:
#get count length with xed

gen_d_xed = mrgd[['act_detail','num_hh_reached', 'num_ben_reached']].groupby(['act_detail'])\
        .agg({'num_hh_reached' : ['sum'], 'num_ben_reached' : ['sum'], 'act_detail' : 'count'}).reset_index()

gen_d_xed.columns = ['_'.join(col) for col in gen_d_xed.columns.values]

gen_d_xed.sort_values('num_ben_reached_sum', ascending=False)

sum(gen_d_xed.act_detail_count)

40567

In [ ]:
#counts by dist area (not really useful)

d = mrgd[['act_cat','num_hh_reached', 'num_ben_reached']].groupby(['act_cat'])\
        .agg({'num_hh_reached' : ['sum'], 'num_ben_reached' : ['sum'], 'act_cat' : 'count'}).reset_index()

d.columns = ['_'.join(col) for col in d.columns.values]

d.sort_values('num_ben_reached_sum', ascending=False)


In [113]:
#data completeness

b = pd.DataFrame(xed.isnull().sum()).reset_index()

b['pct'] = (1 - (b[0]/len(mrgd))) * 100

b.to_clipboard()

In [ ]:
###data completeness by report

#first get number of rows per report
tot_cnts = mrgd.uid.value_counts()

#then get num of blank cells per report
cols = list(xed.columns) + ['uid']
cols.remove('mrg_v_ed')
cols.remove('file_nm')

null_cnt = {}

UID_COL = 'uid'
cnt = 0
def find_empt(r):
    global cnt
    cnt += 1
    if cnt % 1000 == 0:
        print(cnt)
            
    uid = r[UID_COL]
    r.drop(UID_COL)
    
    num_empt = sum(r.isnull())
    
    if r[UID_COL] in null_cnt:
        null_cnt[r[UID_COL]] += num_empt
        
    else:
        null_cnt[r[UID_COL]] = num_empt
    
mrgd.apply(lambda x : find_empt(x[cols]), axis = 1)

In [ ]:
#...continuation

#now merge
ordr = OrderedDict(null_cnt)
msg_bdown = pd.DataFrame(tot_cnts).reset_index()
msg_cnt = pd.DataFrame({'index' : list(ordr.keys()), 'msg_vals' : list(ordr.values())}).reset_index()

msg_bdown = msg_bdown.merge(msg_cnt, on = 'index')
msg_bdown = msg_bdown.rename({'uid' : 'num_row'}, axis = 1)

#-1 for UID
msg_bdown['num_vals'] = msg_bdown.num_row * (len(cols) - 1)
msg_bdown['pct_missing'] = msg_bdown.msg_vals / msg_bdown.num_vals

msg_bdown = msg_bdown.merge(mrgd[['uid', 'country', 'year']].drop_duplicates(), left_on = 'index', right_on='uid', how = 'inner')

msg_bdown = msg_bdown[~msg_bdown.country.isna()]

msg_bdown.to_clipboard()